# Bootstrapping Hazard Rates

## Import Libraries


In [ ]:
#Libraries
import math
import datetime as dt
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import dateutil.relativedelta as relativedelta
from typing import List
import scipy.optimize
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#Import RiVaPy
import rivapy
from rivapy.marketdata import DiscountCurve, DatedCurve, SurvivalCurve
from rivapy import enums
import numpy

## Introduction

In this Notebook we introduce the basic principles of hazard-rates bootstrapping. 

In order to receive hazard rates from the market, one can use a bootstrap methodolody as an iterative approach. By starting with the CDS with the shortest maturity and adding the CDS with the next shortest maturity, all swaps and their hazard rates can be added to the term structure. 

## Credit Curve Bootstrapping Methodology


O'Kane and Turnbull describing the calibration of hazard rates to CDS. Therefore one need to define a standard Credit Default Swap. In this CDS one side pays a premium to receive protection for the underlying asset (premium leg). The premium is payed with a certain frequency until the CDS matures or a default occurs. The other side in the CDS exchanges the recovery value in case a default occurs (protection leg).

The valuation and derivation of the premium leg and the protection leg are described in detail in the Credit Default Swap valuation Notebook. (Link zu anderem Notebook) In order to valuate both legs a Survival Curve is needed.

One can use in the Par-Spread-Formulathe market given parameter and the fair value of CDS with different maturities in order to find the survival rate/hazard rates which is included in the market spreads.

**Par Spread:**

The par spread is the spread that makes the value of a credit default swap with same maturity (and of course with all other details except the coupon identical) zero at present. Therefore the par spread discribes the coupon which would make the two legs of the CDS (protection und premium leg) equal.

The par spread can be approximated by the following formula:

$$S_p(T)=\frac{PV_{Protection Leg}}{PV_{Premium Leg}+PV_{accrued interest}}$$

With the relevant variables:

$$S_p(T)=\frac{(1-RR)*\sum_{i=0}^TP(t_i)(Q(t_{i-1})-Q(t_i))}{\sum_{i=0}^T\Delta_iP(t_i)\frac{Q(t_{i-1}+Q(t_i)}{2}}$$

**Notation:**

$RR=$ Recovery rate of the reference instrument

$P(t)=$ Price of a zero-coupon bond (Risk-free curve)

$t_i=$ specific payment times 

$Q(t)=$ Probabilty of surviving derived by the hazard rates, which can be bootstrapped from the traded credit spreads

$M=$ remaining payments 

$\Delta_i=$ year fractions (between payment dates)


### Calibration of the credit curve

#### Market spreads

In order to understand the bootstrapping methodolody in the following section some examples are shown. Therefore the recover rate (RR) is constant. For CDS with the maturities 1y, 2y and 3y market spreads are given. The coupon is paid yearly, therefore the year fraction (df) is always one.

In [ ]:
#Instrument and market data
RR=0.4
ref_date=dt.datetime(2021,4,30)
#market spreads
sp_1=0.0003
sp_2=0.0009
sp_3=0.0015

#year fraction
df1=1
df2=1
df3=1


marketdata = {"Tenor": [0,1,2,3],
           "Spread": [0,0.0003,0.0009,0.0015]
}

marketdata=pd.DataFrame(marketdata)

print(marketdata)


In this context, the basis points indicated represent the insurance premium that the policyholder has to pay in order to insure himself against a default of the debt securities of the respective company. These premiums can provide even more timely and accurate information about a company's creditworthiness than some ratings. As a general rule, a low CDS - i.e. a low risk premium - indicates a high credit rating, and vice versa.

#### Sample calculation

##### T = 0

We know that the surival probability at T0 is eqaul to 1, because the underlying is not in default at the beginning.

In [ ]:
#survival value at t=0
q_t0=1
duedate=dt.datetime(2021,4,30)
h_t0=0
print("The hazard rate at t=0:",round(h_t0,4),"%")
print("The survival rate at t=0:", round(q_t0*100,4),"%")

In [ ]:
#plot
example = {"tenor": [0,1,2,3,5,7,10,15,20],
       "SurvivalRate": [q_t0*100,'','','','','','','',''],
       "HazardRates": [h_t0,'','','','','','','',''],
           "Spread": [0,'','','','','','','','']
}

example=pd.DataFrame(example)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=example.tenor, y=example.SurvivalRate, name="Survival Rate (in percentage)",marker_color='rgb(158,202,225)', 
        opacity=0.7),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.HazardRates, name="Hazard Rate (in percentage)"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.Spread, name="Spread (in percentage)", marker_color='blue'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hazard Rates Bootstrapped T0"
)

# Set x-axis title
fig.update_xaxes(title_text="Maturity (in years)")

# Set y-axes titles
fig.update_yaxes(title_text="Survival Rate (in %)", secondary_y=False)
fig.update_yaxes(title_text="Hazard Rate (in %)", secondary_y=True, range=[0, 0.8])

fig.show()

##### T = 1

By using the survival probality at T0 and the given market spreads one can derive the Survival rate at **T1**. One has to find $Q(t_1)$ in the par spread formula which equals the left and right side. By rearranging the formula one can derive the value of $Q(t_1)$ with the following formula:

Formula:

$$S_p(1)=\frac{(1-RR)*P(t_1)(Q(t_{0})-Q(t_1))}{\Delta_1P(t_1)\frac{Q((t_0)+Q(t_1)}{2}}$$

We're looking for $Q(t_1)$. 

In [ ]:
#Calculate survival rate at t=1 based on market spreads and survival rate at t=0
duedate_1=dt.datetime(2022,4,30)
q_t1=(q_t0*((1-RR)-sp_1*df1*0.5))/(sp_1*df1*0.5+(1-RR))
h_t1=-numpy.log(q_t1*((duedate_1-ref_date).days/365))*100
print("The hazard rate at t=1:",round(h_t1,2), "%")
print("The survival rate at t=1:", round(q_t1*100,2),"%")

In [ ]:
#plot
example = {"tenor": [0,1,2,3,5,7,10,15,20],
       "SurvivalRate": [q_t0*100,q_t1*100,'','','','','','',''],
       "HazardRates": [h_t0,h_t1,'','','','','','',''],
           "Spread": [0,sp_1*100,'','','','','','','']
}

example=pd.DataFrame(example)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=example.tenor, y=example.SurvivalRate, name="Survival Rate (in percentage)", marker_color='rgb(158,202,225)', 
        opacity=0.7),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.HazardRates, name="Hazard Rate (in percentage)"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.Spread, name="Spread (in percentage)", marker_color='blue'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hazard Rates Bootstrapped T1"
)

# Set x-axis title
fig.update_xaxes(title_text="Maturity (in years)")

# Set y-axes titles
fig.update_yaxes(title_text="Survival Rate (in %)", secondary_y=False)
fig.update_yaxes(title_text="Hazard Rate (in %)", secondary_y=True, range=[0, 0.8])

fig.show()

##### T = 2

By using the survival probality at T0 and T1 and the given market spreads one can derive the survival rate at **T2**. One has to find $Q(t_2)$ in the par spread formula which equals the left and right side:

Formula:

$$S_p(2)=\frac{(1-RR)*[P(t_1)h(t_1)+P(t_2)(Q(t_{1})-Q(t_2))]}{Delta_1P(t_1)\frac{Q(t_0)+Q(t_1)}{2}+Delta_2P(t_2)\frac{Q(t_1)+Q(t_2)}{2}}$$

We're looking for $Q(t_2)$. 

In [ ]:
#Calculate survival rate at t=2 based on market spreads and survival rate at t=1
duedate_2=dt.datetime(2023,4,30)
q_t2=((1-RR)*(q_t0-q_t1)-sp_2*df1*(q_t0+q_t1)*0.5-sp_2*df2*0.5*q_t1+(1-RR)*q_t1)/(sp_2*df2*0.5+(1-RR))
h_t2=-numpy.log(q_t2/q_t1*((duedate_1-ref_date).days/365))*100
print("The hazard rate at t=2:",round(h_t2,2), "%")
print("The survival rate at t=2:", round(q_t2*100,2), "%")

In [ ]:
#plot
example = {"tenor": [0,1,2,3,5,7,10,15,20],
       "SurvivalRate": [q_t0*100,q_t1*100,q_t2*100,'','','','','',''],
       "HazardRates": [h_t0,h_t1,h_t2,'','','','','',''],
           "Spread": [0,sp_1*100,sp_2*100,'','','','','','']
}

example=pd.DataFrame(example)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=example.tenor, y=example.SurvivalRate, name="Survival Rate (in percentage)",marker_color='rgb(158,202,225)', 
        opacity=0.7),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.HazardRates, name="Hazard Rate (in percentage)"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.Spread, name="Spread (in percentage)", marker_color='blue'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hazard Rates Bootstrapped T2"
)

# Set x-axis title
fig.update_xaxes(title_text="Maturity (in years)")


# Set y-axes titles
fig.update_yaxes(title_text="Survival Rate (in %)", secondary_y=False)
fig.update_yaxes(title_text="Hazard Rate (in %)", secondary_y=True, range=[0, 0.8])

fig.show()

##### T = 3

By using the survival probality at T0, T1 and T2 and the given market spreads one can derive the survival rate at **T3**. One has to find $Q(t_3)$ in the par spread formula which equals the left and right side:

Formula:

$$S_p(2)=\frac{(1-RR)*[P(t_1)h(t_1)+P(t_2)h(t_2)+P(t_3)(Q(t_{2})-Q(t_3))]}{Delta_1P(t_1)\frac{Q(t_0)+Q(t_1)}{2}+Delta_2P(t_2)\frac{Q(t_1)+Q(t_2)}{2}+Delta_3P(t_3)\frac{Q(t_2)+Q(t_3)}{2}}$$

We're looking for $Q(t_3)$. 

In [ ]:
#Calculate survival rate at t=3 based on market spreads and survival rate at t=2
duedate_3=dt.datetime(2024,4,30)
q_t3=((1-RR)*(q_t0-q_t1)+(1-RR)*(q_t1-q_t2)+(1-RR)*q_t2-sp_3*df3*0.5*(q_t0+q_t1)-sp_3*df3*0.5*(q_t1+q_t2)-sp_3*df3*0.5*q_t2)/(sp_3*df3*0.5+(1-RR))
h_t3=-numpy.log(q_t3/q_t2*((duedate_1-ref_date).days/365))*100
print("The hazard rate at t=3:",round(h_t3,2), "%")
print("The survival rate at t=3:", round(q_t3*100,2), "%")

In [ ]:
#plot
example = {"tenor": [0,1,2,3,5,7,10,15,20],
       "SurvivalRate": [q_t0*100,q_t1*100,q_t2*100,q_t3*100,'','','','',''],
       "HazardRates": [h_t0,h_t1,h_t2,h_t3,'','','','',''],
           "Spread": [0,sp_1*100,sp_2*100,sp_3*100,'','','','','']
}

example=pd.DataFrame(example)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=example.tenor, y=example.SurvivalRate, name="Survival Rate (in percentage)",marker_color='rgb(158,202,225)', 
        opacity=0.7),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.HazardRates, name="Hazard Rate (in percentage)"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=example.tenor, y=example.Spread, name="Spread (in percentage)", marker_color='blue'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hazard Rates Bootstrapped T3"
)

# Set x-axis title
fig.update_xaxes(title_text="Maturity (in years)")

# Set y-axes titles
fig.update_yaxes(title_text="Survival Rate (in %)", secondary_y=False)
fig.update_yaxes(title_text="Hazard Rate (in %)", secondary_y=True, range=[0, 0.8])

fig.show()

## Calculate Hazard Rates for specific Market Data

### Define Functions

In [ ]:
#function to calculate par spread based on instrument and market data
def par_spread(ref_date, trade_date, dc_survival, payment_dates, maturity_date, dc, RR):
    integration_step= relativedelta.relativedelta(days=365)
    premium_period_start = ref_date
    prev_date=ref_date
    current_date=min(prev_date+integration_step, maturity_date)
    dc_valuation_date=dc.value(ref_date, maturity_date)
    risk_adj_factor_protection=0
    risk_adj_factor_premium=0
    risk_adj_factor_accrued=0
    #RR=0.4

    while current_date <= maturity_date:
        default_prob = dc_survival.value(ref_date, prev_date)-dc_survival.value(ref_date, current_date)
        #print("default_prob:",default_prob)
        risk_adj_factor_protection += dc.value(ref_date, current_date) * default_prob
        #print("risk_adj_factor_protection:",risk_adj_factor_protection)
        prev_date = current_date
        current_date += integration_step
    
    if prev_date < maturity_date and current_date > maturity_date:
        default_prob = dc_survival.value(ref_date, prev_date)-dc_survival.value(ref_date, maturity_date)
        #print("default_prob:",default_prob)
        risk_adj_factor_protection += dc.value(ref_date, maturity_date)  * default_prob
        #print("risk_adj_factor_protection:",risk_adj_factor_protection)


    for premium_payment in payment_dates:
        if premium_payment >= ref_date:
            period_length = ((premium_payment-premium_period_start).days)/360
            #survival_prob = (dc_survival.value(ref_date, premium_period_start)+dc_survival.value(ref_date, premium_payment))/2
            survival_prob = dc_survival.value(ref_date, premium_period_start)
            #print("survival_prob:",survival_prob)
            df = dc.value(ref_date, premium_payment)
            risk_adj_factor_premium += period_length*survival_prob*df
            #print("risk_adj_factor_premium:",risk_adj_factor_premium)
            default_prob = dc_survival.value(ref_date, premium_period_start)-dc_survival.value(ref_date, premium_payment)
            #print("default_prob_accrued:",default_prob)
            risk_adj_factor_accrued += period_length*default_prob*df
            #print("risk_adj_factor_accrued:",risk_adj_factor_accrued)
            premium_period_start = premium_payment



    PV_accrued=((1/2)*risk_adj_factor_accrued)
    #print("PV_accrued: ",PV_accrued)
    PV_premium=(1)*risk_adj_factor_premium
    #print("PV_premium: ",PV_premium)
    PV_protection=(((1-RR))*risk_adj_factor_protection)
    #print("PV_protection: ",PV_protection)
    
    par_spread_i=(PV_protection)/((PV_premium+PV_accrued))
    #print("par_spread_i: ",par_spread_i)
    return par_spread_i

In [ ]:
#Function to Rollout payment_dates
def payment_dates(trade_date,maturity_date,payment_cycle):
    date=maturity_date-relativedelta.relativedelta(months=+payment_cycle)
    payment_dates=[maturity_date]
    if maturity_date<date:
       payment_dates.append(maturity_date)
    while date>=trade_date:
        payment_dates.append(date)
        date=date-relativedelta.relativedelta(months=+payment_cycle)
    return sorted(payment_dates)
#payment_dates_ins=payment_dates(trade_date,maturity_date,payment_cycle)

In [ ]:
#Bootstrapping Functionality
def create_survival(refdate, dates: List[dt.datetime], hazard_rates: List[float]):
    return SurvivalCurve('survival_curve', refdate, dates, hazard_rates)
    
def calibration_error(x, mkt_par_spread, ref_date, trade_date, payment_dates, dc_new, dates, hazard_rates, RR):
    hazard_rates[-1] = x
    maturity_date = dates[-1]
    dc_surv = create_survival(ref_date, dates, hazard_rates)
    #print(x)
    #print("Market Spread=",mkt_par_spread,"-","Par-Spread=",par_spread(ref_date, trade_date, dc_surv, payment_dates,maturity_date, dc_new),"=",
    #      mkt_par_spread - par_spread(ref_date, trade_date, dc_surv, payment_dates,maturity_date, dc_new))
    return  mkt_par_spread - par_spread(ref_date, trade_date, dc_surv, payment_dates, maturity_date, dc_new, RR)


def calibrate_hazard_rate(payment_dates_bootstrapp, ref_date,mkt_par_spread, trade_date, dc_new, RR):
    #sort payment_dates by [-1]
    sc_dates=[ref_date]
    hazard_rates=[0.0]
    for i in range(len(payment_dates_bootstrapp)):
        payment_dates_iter = payment_dates_bootstrapp[i]
        #print(payment_dates_iter)
        mkt_par_spread_iter = mkt_par_spread[i]
        #print(mkt_par_spread_iter)
        sc_dates.append(payment_dates_iter[-1])
        hazard_rates.append(hazard_rates[-1])
        sol=scipy.optimize.root_scalar(calibration_error,args=(mkt_par_spread_iter, ref_date, trade_date, 
                         payment_dates_iter, dc_new, sc_dates, hazard_rates, RR),method='brentq',bracket=[0,3],xtol=1e-8,rtol=1e-8)
        hazard_rates[-1] = sol.root
        #print(numpy.exp(-sol.root*((payment_dates_iter[-1]-ref_date).days/365)))
    return create_survival(ref_date, sc_dates, hazard_rates), hazard_rates



### Define market data

In [ ]:
#Define yield curve
object_id = "CDS_interest_rate_boots"
refdate = dt.datetime(2020,1,1)
days_to_maturity = [360, 720, 3*360, 4*360, 5*360, 7*360, 10*360,  15*360, 20*360,  30*360]
dates = [refdate + timedelta(days=d) for d in days_to_maturity]
rates = [0.002585,0.005034,0.008981,0.012954,0.016452,
                   0.021811,0.027007,0.031718,0.033834,0.035056]
#rates = [0,0,0,0,0,0,0,0,0,0]
dsc_fac = [math.exp(-rates[i]*days_to_maturity[i]/360) for i in range(len(days_to_maturity))]

        
dc_new = DiscountCurve(object_id, refdate, dates, dsc_fac, enums.InterpolationType.LINEAR, enums.ExtrapolationType.LINEAR)

In [ ]:
#Market Spreads
#Pfizer
mkt_par_spread=[0.0003,0.0009,0.0015,0.0021,0.0028,0.0043,0.0061,0.0063, 0.0068,0.0066]

#Radioshack
mkt_par_spread_rs=[0.6405,0.5956,0.5511,0.5144,0.4894,0.4511,0.4156,0.3815,0.3657,0.3506]



In [ ]:
#dataframe
marketdata_test = {"Tenor": [1,2,3,4,5,7,10,15,20,30],
           "Spread Pfizer": mkt_par_spread,
                   "Spread Radioshack" : mkt_par_spread_rs
}

marketdata_test=pd.DataFrame(marketdata_test)

print(marketdata_test)

### Define Instrument data

In [ ]:
#Instrument data
refdate = dt.datetime(2020,1,1)
trade_date = refdate
RR=0.4
RR_rs=0.25
maturities_CDS_test = [12,24,36,48,60,7*12,10*12,15*12,20*12,30*12]

#Maturities and Payment Dates
#ref_date=dt.datetime(2020,1,1)
maturity_dates=[]
date=refdate
for i in range (len(maturities_CDS_test)):
    if maturities_CDS_test[i]>=12:
        date=refdate+relativedelta.relativedelta(years=(maturities_CDS_test[i]/12))
        maturity_dates.append(date)
    elif maturities_CDS_test[i]<12:
        date=refdate+relativedelta.relativedelta(month=maturities_CDS_test[i])
        maturity_dates.append(date)
    
maturity_dates_sorted=sorted(maturity_dates)

#Rollout instrument payment dates for different maturities
payment_dates_bootstrapp=[]
for i  in range(len(maturity_dates_sorted)):
    payment_dates_bootstrapp.append(payment_dates(refdate,maturity_dates_sorted[i],3))
payment_dates_bootstrapp=sorted(payment_dates_bootstrapp, key=lambda x: x[-1])


### Bootstrapp Survival Curve

#### Pfizer

In [ ]:
#Create Survival Curve based on Bootstrapping alogirthm for Pfizer
survival_curve=calibrate_hazard_rate(payment_dates_bootstrapp, refdate,mkt_par_spread, trade_date, dc_new, RR)
survival_curve=survival_curve[0]
survival_curve_bootstrap=[survival_curve.value(refdate,refdate)]

for i in range(len(payment_dates_bootstrapp)):
    survival_curve_bootstrap.append(survival_curve.value(refdate,payment_dates_bootstrapp[i][-1]))
del survival_curve_bootstrap[0]
#print(survival_curve_bootstrap)

##### Get Data for Plots

In [ ]:
#Hazard Rates and survival rates for plot
hazard_rates=calibrate_hazard_rate(payment_dates_bootstrapp, refdate,mkt_par_spread, trade_date, dc_new, RR)[1]
del hazard_rates[0]
hazard_rates=[rate*100 for rate in hazard_rates]
#print(hazard_rates)
#Survival Rates for plot
survival_rates=survival_curve_bootstrap
survival_rates=[rate*100 for rate in survival_rates]
x=[maturities/12 for maturities in maturities_CDS_test]
#print(survival_rates)

In [ ]:
#Reference Data
test= {"tenor": [1,2,3,4,5,7,10,15,20,30],
       "SurvivalRate": [99.95, 99.70, 99.25, 98.59, 97.65, 94.92, 89.69, 84.71, 78.37, 71.27],
       "HazardRates": [0.05, 0.25, 0.45, 0.66, 0.96, 1.42, 1.89, 1.14, 1.56, 0.95]
}
refDataPfizer=pd.DataFrame(test)
#print(refDataPfizer)

##### Plot

In [ ]:
#Plotting Hazard Rates for different maturities
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=x, y=survival_rates, name="Survival Rate (in percentage)",marker_color='blue', 
        opacity=0.5),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=x, y=refDataPfizer.SurvivalRate, name="Survival Rate Reference (in percentage)",marker_color='green', 
        opacity=0.5),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=x, y=hazard_rates, name="Hazard Rate (in percentage)", marker_color='brown'),
    secondary_y=True,
)



fig.add_trace(
    go.Scatter(x=x, y=refDataPfizer.HazardRates, name="Hazard Rate (in percentage)", marker_color='black'),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Hazard and Survival Rates Bootstrapped (Pfizer)"
)

# Set x-axis title
fig.update_xaxes(title_text="Maturity (in years)")

# Set y-axes titles
fig.update_yaxes(title_text="Survival Rate (in %)", secondary_y=False)
fig.update_yaxes(title_text="Hazard Rate (in %)", secondary_y=True)

fig.show()

#### Radioshack

In [ ]:
#Create Survival Curve based on Bootstrapping alogirthm for Pfizer
survival_curve_rs=calibrate_hazard_rate(payment_dates_bootstrapp, refdate, mkt_par_spread_rs, trade_date, dc_new, RR_rs)
survival_curve_rs=survival_curve_rs[0]
survival_curve_bootstrap_rs=[survival_curve_rs.value(refdate,refdate)]

for i in range(len(payment_dates_bootstrapp)):
    survival_curve_bootstrap_rs.append(survival_curve_rs.value(refdate,payment_dates_bootstrapp[i][-1]))
del survival_curve_bootstrap_rs[0]
#print(survival_curve_bootstrap_rs)

##### Get Data for Plots

In [ ]:
#Hazard Rates and survival rates for plot
hazard_rates_rs=calibrate_hazard_rate(payment_dates_bootstrapp, refdate, mkt_par_spread_rs, trade_date, dc_new, RR_rs)[1]
del hazard_rates_rs[0]
hazard_rates_rs=[rate*100 for rate in hazard_rates_rs]
#print(hazard_rates)
#Survival Rates for plot
survival_rates_rs=survival_curve_bootstrap_rs
survival_rates_rs=[rate*100 for rate in survival_rates_rs]
x=[maturities/12 for maturities in maturities_CDS_test]
#print(survival_rates)

In [ ]:
#Reference Data
test= {"tenor": [1,2,3,4,5,7,10,15,20,30],
       "SurvivalRate": [34.18, 15.38, 10.22, 8.45, 6.95, 5.66, 4.39, 3.37, 2.13, 1.36],
       "HazardRates": [107.37, 79.84, 40.91, 18.95, 19.61, 10.22, 8.51, 5.29, 9.16, 4.48]
}
refDataRS=pd.DataFrame(test)
#print(refDataRS)

##### Plot

In [ ]:
#Plotting Hazard Rates for different maturities
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=x, y=survival_rates_rs, name="Survival Rate (in percentage)",marker_color='blue', 
        opacity=0.5),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=x, y=refDataRS.SurvivalRate, name="Survival Rate Reference (in percentage)",marker_color='green', 
        opacity=0.5),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=x, y=hazard_rates_rs, name="Hazard Rate (in percentage)", marker_color='brown'),
    secondary_y=True,
)


fig.add_trace(
    go.Scatter(x=x, y=refDataRS.HazardRates, name="Hazard Rate (in percentage)", marker_color='black'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hazard and Survival Rates Bootstrapped (Radioshack)"
)

# Set x-axis title
fig.update_xaxes(title_text="Maturity (in years)")

# Set y-axes titles
fig.update_yaxes(title_text="Survival Rate (in %)", secondary_y=False)
fig.update_yaxes(title_text="Hazard Rate (in %)", secondary_y=True)

fig.show()